In [1]:
from utils import Neo4jConnection

In [3]:
con = Neo4jConnection("bolt://localhost:7687","neo4j","academic")

Verbindung zur Neo4j-Datenbank erfolgreich hergestellt.


In [11]:
insert_page_data(con, page_data)

In [8]:
page_data = {
    "page_id": 12,
    "sections": [
        {
            "name": "Introduction",
            "text": ("Anarchism is an anti-authoritarian political and social philosophy that rejects hierarchies "
                     "deemed unjust and advocates their replacement with self-managed, self-governed societies..."),
            "link_lengths": [18, 9, 17, 11, 12, 13, 11, 19, 17, 5, 8, 18, 9, 16, 9, 12, 11, 9, 23, 30, 28, 13, 12, 6, 23],
            "link_offsets": [16, 35, 49, 80, 143, 157, 201, 269, 405, 500, 586, 602, 638, 652, 715, 726, 740, 753, 767, 886, 964, 1052, 1078, 1160, 1171],
            "target_page_ids": [867979, 23040, 586276, 13998, 40949353, 191161, 89313, 4228181, 26271818, 23604120, 18247344, 23490, 1228884, 5483237, 17865, 46399294, 49938, 1799997, 76393, 1063286, 46399274, 15181, 5708, 39353100, 14936]
        }
    ]
}

In [10]:
def insert_page_data(conn, page_data):
    # Cypher-Abfrage für das Einfügen der Seite und Sektionen
    query = """
    WITH $page_data AS data

    MERGE (p:Page {id: data.page_id})

    FOREACH (section IN data.sections | 
      MERGE (s:Section {name: section.name, text: section.text})
      MERGE (p)-[:HAS_SECTION]->(s)
      
      FOREACH (i IN RANGE(0, SIZE(section.target_page_ids)-1) |
        MERGE (target:Page {id: section.target_page_ids[i]})
        MERGE (s)-[:LINKED_TO {offset: section.link_offsets[i], length: section.link_lengths[i]}]->(target)
      )
    )
    """
    conn.run_query(query, {"page_data": page_data})

In [12]:
import json

data = []
with open("../raw_data/link_annotated_text.jsonl") as f:
    for line in f:
        data.append(json.loads(line))
        break

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/link_annoteted_text.jsonl'